In [17]:
import requests
import zipfile
import os
import tempfile
import geopandas as gpd

# URL of the zip file containing the shapefile
url = 'https://github.com/ShengAric92/CASA0013_group_assignment/blob/main/WD_DEC_2022_UK_BGC.zip?raw=true'

# Download the zip file
response = requests.get(url)
zip_file_path = tempfile.mktemp(suffix=".zip")

with open(zip_file_path, 'wb') as f:
    f.write(response.content)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    temp_dir = tempfile.mkdtemp()
    zip_ref.extractall(temp_dir)

# Find the shapefile and read it using geopandas
for file in os.listdir(temp_dir):
    if file.endswith(".shp"):
        shapefile_path = os.path.join(temp_dir, file)
        wards = gpd.read_file(shapefile_path)
        break

In [18]:
import pandas as pd
import requests
from io import StringIO

# URL of the CSV file
url = 'http://data.insideairbnb.com/united-kingdom/england/london/2023-09-06/visualisations/listings.csv'

# Download the CSV file
response = requests.get(url)
response.raise_for_status()  # This will raise an error if the download failed

# Read the CSV file into a Pandas DataFrame
csv_string = StringIO(response.text)
listings = pd.read_csv(csv_string)

/tmp/ipykernel_46850/571644261.py:14: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  listings = pd.read_csv(csv_string)


In [19]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd

# Assuming listings is a list of dictionaries, convert it to a DataFrame
listings_df = pd.DataFrame(listings)

# Convert listings latitude and longitude to Point objects
listings_df['geometry'] = listings_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# Convert DataFrame to GeoDataFrame
listings_geo = gpd.GeoDataFrame(listings_df, geometry='geometry')

# Set the coordinate reference system (CRS) to EPSG:4326 (WGS 84)
listings_geo.set_crs(epsg=4326, inplace=True)

# Ensure wards data is in the same CRS as listings_geo
wards = wards.to_crs(epsg=4326)

# Perform the spatial join
listings_with_wardcode = gpd.sjoin(listings_geo, wards, how="left", op='intersects')

# Select columns to display
##The column name may change,ESPECITALLY WD22CD AND WD22NM, SO PAY ATTENTION!
columns_to_display = ['id', 'name', 'latitude', 'longitude', 'WD22CD', 'WD22NM']
result = listings_with_wardcode[columns_to_display]

# Display the result
print(result.head())

      id                                               name  latitude  \
0  13913  Rental unit in Islington · ★4.80 · 1 bedroom ·...  51.56861   
1  15400  Rental unit in London · ★4.80 · 1 bedroom · 1 ...  51.48780   
2  92644  Rental unit in Earlsfield · ★4.57 · 1 bedroom ...  51.44201   
3  17402  Rental unit in London · ★4.76 · 3 bedrooms · 3...  51.52195   
4  93015  Rental unit in Hammersmith · ★4.82 · 2 bedroom...  51.49993   

   longitude     WD22CD             WD22NM  
0   -0.11270  E05013712         Tollington  
1   -0.16813  E05009405            Stanley  
2   -0.18739  E05014027  Wandsworth Common  
3   -0.14094  E05013808           West End  
4   -0.21707  E05013735        Brook Green  


/opt/conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [14]:
listings_with_ward.to_csv('listings_with_wardcode.csv', index=False)